In [5]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Load packages and initialize GEE

In [6]:
import ee, datetime
import pandas as pd
import matplotlib.dates as mdates
import csv
import random as random_local

from IPython.display import Image, display, HTML

from matplotlib import dates
from pylab import *

import ipywidgets as widgets

ee.Initialize()

print('Packages loaded')

Packages loaded


## Load the FRA RSS tile footprints

In [7]:
# Get all the tiles footprint
world_tiles_cd = ee.FeatureCollection('users/kindgard/RSS2020/worldtilesALL_cd2')

# Call country boundaries
countries = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw')
list_countries = ee.List(countries.aggregate_array('Country')).getInfo()

w_country = widgets.Dropdown(
    options=list_countries,
    description='Country:',
    disabled=False
)

w_nbSamples = widgets.IntSlider(
    value=200,
    min=0,
    max=500,
    step=50,
    description='Samples :'
)



## Select country and number of samples

In [16]:
display(w_country)
display(w_nbSamples)

Dropdown(description='Country:', index=208, options=('Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',…

IntSlider(value=100, description='Samples :', max=500, step=50)

## Identify tiles overlapping with AOI

In [18]:
## Select country and number of samples
country_name = w_country.value
size         = w_nbSamples.value

the_country = countries.filterMetadata('Country', 'equals', country_name)

aoi = the_country.geometry().buffer(5000)

tiles = world_tiles_cd.filterBounds(aoi)

list_tiles = ee.List(tiles.aggregate_array('Source')).getInfo()

i=1

for tile in list_tiles:
    print(tile)
    for row in csv.reader(open('/home/contessa/fra_sepal/data/list_feature_collection_20190905.csv','r')):
        if (row[3] == tile):
            print(row[0])
            if (i==1):
                collection = ee.FeatureCollection(row[0])
            else:
                collection = collection.merge(ee.FeatureCollection(row[0]))
            i=i+1

Africa2_10
users/kindgard/DGGRID/Africa2_grid10
Africa2_2
users/kindgard/DGGRID/Africa2_grid2
Africa2_9
users/kindgard/DGGRID/Africa2_grid9


## Get the list of available samples

In [19]:
list_avail  = []
list_change = []
list_nochfo = []
list_nochnf = []
list_smallc = []
sample_list = []

for row in csv.reader(open('../data/world_fra_samples_20190404.csv','r')):
        if (row[7] == country_name):
            list_avail.append(row[3])
            if (row[5] == "change"):
                list_change.append(row[3])
            elif (row[5] == "smallchange"):
                list_smallc.append(row[3])
            elif (row[5] == "nochangeforest"):
                list_nochfo.append(row[3])
            else:
                list_nochnf.append(row[3])

s = math.floor(size/4)

print(s)
print(len(list_avail))

print(len(list_change))
print(len(list_smallc))
print(len(list_nochfo))
print(len(list_nochnf))

change_list = random_local.sample(list_change,min(len(list_change),s))
smallc_list = random_local.sample(list_smallc,min(len(list_smallc),s))
nochfo_list = random_local.sample(list_nochfo,min(len(list_nochfo),s))
nochnf_list = random_local.sample(list_nochnf,min(len(list_nochnf),s))

sample_list.extend(change_list+smallc_list+nochfo_list+nochnf_list)

samples = collection.filterBounds(aoi).filter(ee.Filter.inList('global_id',sample_list))

print(len(sample_list))

25
1543
196
155
963
229
100


In [16]:
#def getCentroid(featureCollection):
#    for feature in featureCollection
#        centroid = feature.centroid()
#        square = centroid.buffer(50).bounds().geometry()
#        return ee.Feature(square,{global_id: feature.get('global_id')})
#    return 
 
#centroids = getCentroid(samples)
 
#samplesWithCentroids= samples.merge(centroids) 


### Export results as KML in your Google Drive

In [ ]:
task = ee.batch.Export.table.toDrive(collection  = samples,
                                     description = 'task_samples',
                                     fileFormat  = 'SHP',
                                     fileNamePrefix = 'missing_samples_'+country_name)
task.start()

state = task.status()['state']
while state in ['READY', 'RUNNING']:
    print(state + '...')
    time.sleep(5) 
    state = task.status()['state']
print('Done.', task.status())
print(task.status()['state'])

READY...
READY...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
READY...
READY...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
READY...
READY...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
RUNNING...
